<a href="https://colab.research.google.com/github/adityasahusomu/Youtube_Comment_Analyzer/blob/main/experiment_3_tfidf_trigram_max_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install mlflow boto3 awscli

In [15]:
import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [16]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': 'F35R6HYEEPP5C2QR', 'HostId': 'TWBLifxACfTI8SIbz48uF5hjHlPbtjLDPXOig1tQSb4g7J0PlxnaWmLZ9cBD355Rc+Ny1mv2twCKgm9bKnKJxC7UxKCvBudT8Y/AO5D8bTQ=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'TWBLifxACfTI8SIbz48uF5hjHlPbtjLDPXOig1tQSb4g7J0PlxnaWmLZ9cBD355Rc+Ny1mv2twCKgm9bKnKJxC7UxKCvBudT8Y/AO5D8bTQ=', 'x-amz-request-id': 'F35R6HYEEPP5C2QR', 'date': 'Wed, 15 Oct 2025 12:54:46 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [20]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

In [21]:
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [22]:
mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")

2025/10/15 12:56:13 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://project1-mlflow-bucket/534531239366766743', creation_time=1760532973592, experiment_id='534531239366766743', last_update_time=1760532973592, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [24]:
df = pd.read_csv('/content/cleaned_reddit_dataset.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [25]:
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/10/15 12:59:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/15 12:59:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743/runs/b8c274c9dad147c68980954961f332d6
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743


2025/10/15 13:00:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/15 13:00:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743/runs/08dd1c7123e0427eb67a6776d857e3ff
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743


2025/10/15 13:00:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/15 13:00:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743/runs/f338493593244242bbd70cbf8e817060
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743


2025/10/15 13:03:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/15 13:03:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743/runs/c05c7ac3dda641099307a91f2b1964a5
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743


2025/10/15 13:03:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/15 13:03:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743/runs/58aede96d04e459cb84e55c975172851
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/534531239366766743
